In [ ]:
import open3d as o3d
import numpy as np
import pandas as pd
import os
import laspy
import scipy
import matplotlib.pyplot as plt
from scipy.interpolate import griddata,UnivariateSpline
from scipy.stats import zscore
from laspy.compression import LazrsBackend 
import geopandas as gpd
import src.gvl.trunk_utils as utils
from scipy.spatial import KDTree, distance,ConvexHull
import geopandas as gpd
from shapely.geometry import Polygon,Point



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [10]:
# Change to the appropriate directory
os.chdir('C:/Users/User/Desktop/Workstation/Internship/Data/LiDAR/AHN4/Tree_Detection_in_Aerial_Point_Clouds/notebooks') #Kavel__186_428_sample_1_reclassified.las"

In [ ]:
#Step 1: read the las file 
las_file = laspy.read("../datasets/AHN4/AMS_subtiles_1000/ahn4_186_428_sample_1.las", laz_backend=[LazrsBackend()]) #ahn4_186_428_sample_1.las#Kavel__186_428_sample_1_reclassified.las
np.unique(las_file.classification)

array([2, 5, 6], dtype=uint8)

In [13]:
# Step 2: Filter ground points (class 2)
ground_mask = las_file.classification == 2 #for Kavel_10 class 5 is tree
ground_points = np.c_[las_file.x[ground_mask], las_file.y[ground_mask], las_file.z[ground_mask]]

# Step 3: Interpolate ground elevation (Digital Elevation Model - DEM)
# Define grid for interpolation based on the spatial extent of the LiDAR data
grid_x, grid_y = np.meshgrid(
    np.linspace(np.min(las_file.x), np.max(las_file.x), 500),
    np.linspace(np.min(las_file.y), np.max(las_file.y), 500)
)

# Interpolate the ground surface (DEM) using griddata
grid_z = griddata(ground_points[:, :2], ground_points[:, 2], (grid_x, grid_y), method='linear')

# Flatten the grid for easier subtraction later
grid_points = np.c_[grid_x.ravel(), grid_y.ravel()]
grid_z_flat = grid_z.ravel()

# Step 4: Normalize the entire point cloud by subtracting ground elevation
all_points = np.c_[las_file.x, las_file.y, las_file.z]
normalized_z = las_file.z - griddata(grid_points, grid_z_flat, (las_file.x, las_file.y), method='linear')

# Replace the original Z values with the normalized Z values
las_file.z = normalized_z

In [14]:
# Filter points classified as 'tree'
tree_mask = las_file.classification == 5 #for Kavel_10 class 5 is tree

# Create a new LAS object with only 'tree' points
filtered_las = laspy.LasData(las_file.header)
las_file.points = las_file.points[tree_mask]

In [15]:
# Concatenate the file coordinates
coord = np.c_[las_file.x, las_file.y, las_file.z]
coord.shape

(483452, 3)

In [16]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(coord)
pcd_down = pcd.voxel_down_sample(voxel_size=0.3)
coord = np.asarray(pcd_down.points)
coord.shape

(396468, 3)

In [17]:
# Sort the coordinates by z value
position = coord[coord[:, 2].argsort()]

In [18]:
# Create a list of "point" class for each set of coordinates
points = []
for i in range(len(position)):
    i = utils.Point(i, position[i])
    points.append(i)

In [19]:
# Parameters (in meters)
r = 3  # radius of the search sphere for the initial clustering
radius = 0.8  # the radius on which we count the point density in x and y for each point
# (the parameter used for local maxima calculation)
window_size = 4 # the size of the search window for local maxima in each cluster
max_distance = 7.5  # the delineated trunks radius
restrict_d = (
    2  # the minimum eucledian distance that 2 peaks of the same cluster can have
)
small_clusters = 150  # the size of the small custers we suspect as outliers
# (won't be deleted, they will just merge with a nearby big cluster if there is any,
# else they will be taken as individual clusters)
small_outliers = 50 # the minimal cluster size to be allowed as a tree.
# Deleting every cluster below this value (optional).
diff_height = (
    1.5  # the difference in height between 2 clusters very close to each other
)
# (this is the parameter to take care of branches that are classified as a separate cluster)
branch_dist = 0.8  # the max distance a branch cluster can be from the main tree
min_dist_tree = (
    2  # the max distance of 2 clusters to be checked if they are the same tree
)

In [20]:
# Find all points within distance r of point(s) x
tree = scipy.spatial.cKDTree(position)
nn = tree.query_ball_point(position, r)

In [21]:
links = np.zeros(len(position), dtype=int)
centroids = np.zeros((len(position), 3))
has_parent = np.zeros(len(position), dtype=bool)

# Loop over all points
for i, this_nn in enumerate(nn):
    # If the point has no neighbors within radius r, it is a tree peak
    if len(this_nn) == 1:
        links[i] = i
        centroids[i] = position[i]
        has_parent[i] = True
    # If the point has at least one neighbor within radius r
    else:
        # Find all neighbors with a higher z value
        upper_nnbs = [j for j in this_nn if position[j, 2] > position[i, 2]]
        # If there are no such neighbors, the point is a tree peak
        if not upper_nnbs:
            links[i] = i
            centroids[i] = position[i]
            has_parent[i] = True
        # If there are any neighbors with a higher z value
        else:
            # Calculate the centroid of the group of neighbors
            centroids[i] = np.mean(position[upper_nnbs], axis=0)
            # Calculate the distances between each neighbor and the centroid
            dist = scipy.spatial.distance.cdist(
                position[upper_nnbs], [centroids[i]], metric="euclidean"
            )
            # Find the neighbor closest to the centroid and store its index as a link
            links[i] = upper_nnbs[np.argmin(dist)]

has_parent = has_parent.astype("int")

In [22]:

networks = []
all_paths = []
for p in points:
    current_idx = p.index

    if len(points[current_idx].paths) == 0:
        end = False

        # initialize new path
        new_path = utils.Path(len(all_paths))  # len paths as index
        all_paths.append(new_path)

        # add first point to the path
        new_path.add_point(points[current_idx])
        points[current_idx].add_path(new_path)

        # append path
        while end is False:
            # point has a parent
            if has_parent[current_idx] != 1:
                # make link
                points[current_idx].linked_to = points[links[current_idx]]

                if len(points[current_idx].linked_to.paths) == 0:
                    # not in path
                    points[current_idx].linked_to.add_path(new_path)
                    new_path.add_point(points[current_idx].linked_to)
                    current_idx = links[current_idx]

                else:
                    # in path
                    points[current_idx].linked_to.network.add_path(new_path)
                    points[current_idx].add_path(new_path)
                    points[current_idx].linked_to.add_path(new_path)
                    end = True

            # point has no parent
            # make network, end path
            else:
                points[current_idx].linked_to = points[current_idx]
                # init new network
                new_network = utils.Network(len(networks))  # len networks as index
                new_network.add_path(
                    new_path
                )  # path and points are assigned to network
                new_network.top = current_idx
                new_network.points = new_path.points  # add points to the network
                networks.append(new_network)
                points[current_idx].network = new_network
                end = True

4. Remove all the outlier clusters

In [23]:
# Create array to extract and store all our individual tree labels from
labels = np.zeros(len(points))

# Extract the label value from class network to our new built array
for p in points:
    labels[p.index] = p.network.index
labels = labels.astype("int")

array_test = np.column_stack((position, labels))

In [24]:
# Get the count of each cluster label
labels_new = array_test[:, 3]
array = array_test[:, 0:3]

Remove clusters

In [25]:
# Create a dictionary to store the count of each label
unique, counts = np.unique(labels_new, return_counts=True)
label_count = dict(zip(unique, counts))

# Initialize an empty list to store the indices of the large clusters
large_cluster_indices = []

# Iterate through the cluster labels
for i, label in enumerate(labels_new):
    # If the label corresponds to a large cluster, add the index to the list
    if label_count.get(label, 0) >= 10:
        large_cluster_indices.append(i)

# Use the indices of the large clusters to create a new array
array_test = array[large_cluster_indices, :]

# Add the labels as the last column of the new array
array_test = np.column_stack((array_test, labels_new[large_cluster_indices]))

5. Fix the small clusters

In [26]:
# Prepare the array for the "fix small clusters" code
labels_2 = array_test[:, 3].astype("int")
labels33, point_count33 = np.unique(labels_2, return_counts=True)

In [27]:
iterating_array = []
for i in range(len(labels33)):
    if point_count33[i] <= small_clusters:
        iterating_array.append(labels33[i])

In [28]:
# Get centroids of all clusters in the dataset
all_centroids = []
all_labs = []
for label in np.unique(array_test[:, 3]):
    centroid = array_test[array_test[:, 3] == label, :2].mean(axis=0)
    all_centroids.append(centroid)
    all_labs.append(label)

In [29]:
# Find the pairs of the closest clusters
tree1 = KDTree(all_centroids)

labels_nn = []
for i in range(len(all_labs)):
    point_cent = all_centroids[i]
    dist, idx = tree1.query(point_cent, k=2)
    closest_idx = idx[1] if idx[0] == i else idx[0]
    labels_nn.append([all_labs[i], all_labs[closest_idx]])

# Filter the list so it contains only the small clusters that we will fix
filtered_list = [x for x in labels_nn if int(x[0]) in iterating_array]
array_test2 = array_test.copy()

In [30]:
for i in filtered_list:
    coord_xy = array_test2[array_test2[:, 3] == i[0]]
    coord_xy2 = array_test2[array_test2[:, 3] == i[1]]
    wk = distance.cdist(coord_xy[:, :2], coord_xy2[:, :2], "euclidean")
    z = abs(coord_xy[:, 2:3].min() - coord_xy[:, 2:3].min())
    kk = array_test2[:, 2][array_test2[:, 3] == i[1]]
    z = abs(coord_xy[:, 2:3].min() - kk.min())
    if (
        len(array_test2[array_test2 == i[0]]) < (small_clusters / 2)
        and wk.min() < min_dist_tree
    ):
        array_test[:, 3][array_test[:, 3] == i[0]] = i[1]
    if wk.min() < branch_dist and z > diff_height:
        array_test[:, 3][array_test[:, 3] == i[0]] = i[1]
    if (
        len(array_test2[array_test2 == i[0]]) < small_clusters
        and wk.min() < min_dist_tree / 2
    ):
        array_test[:, 3][array_test[:, 3] == i[0]] = i[1]
    coord_xy = []
    coord_xy2 = []
    wk = []
    ind = []

Delete small clusters (optional)

In [31]:
# Get the count of each cluster label
labels_new = array_test[:, 3]
array = array_test[:, 0:3]

# Create a dictionary to store the count of each label
unique, counts = np.unique(labels_new, return_counts=True)
label_count = dict(zip(unique, counts))

# Initialize an empty list to store the indices of the large clusters
large_cluster_indices = []

# Iterate through the cluster labels
for i, label in enumerate(labels_new):
    # If the label corresponds to a large cluster, add the index to the list
    if label_count.get(label, 0) >= small_outliers:
        large_cluster_indices.append(i)

# Use the indices of the large clusters to create a new array
array_test = array[large_cluster_indices, :]

# Add the labels as the last column of the new array
array_test = np.column_stack((array_test, labels_new[large_cluster_indices]))

6. Get the number of points in buffer per point (the local maxima column)

In [32]:
# Input data
points = array_test[:, :2]

# Create KDTree from points
kd_tree = KDTree(points)

# Array to store the number of points in the buffer for each point
count = np.zeros(len(points), dtype=int)

# Loop over each point and find points in the buffer
for i, p in enumerate(points):
    idx = kd_tree.query_ball_point(p, radius)
    count[i] = len(idx) - 1

7. Find the trees

In [33]:
def cluster_local_maxima(full_array, window_size, max_distance, restrict_d):
    # get the unique label of tree clusters
    unique_clusters = np.unique(full_array[:, 3])
    current_label = 1
    labels = np.zeros(full_array.shape[0], dtype=np.int64)
    full_array = np.column_stack((full_array, labels))
    iteration = 0
    # Iterate through every single tree cluster separately
    for cluster_id in unique_clusters:
        peaks1 = []
        dist_peaks = 100
        # Form an array for the cluster of this iteration
        kot_arr = full_array[full_array[:, 3] == cluster_id]
        x1 = kot_arr[:, 0]
        y1 = kot_arr[:, 1]
        z1 = kot_arr[:, 2]
        p1 = kot_arr[:, 4]
        labels_k = kot_arr[:, 5]
        # Now we iterate through each point of the cluster of this iteration
        for i in range(len(kot_arr)):
            # We form a search window around each point of the cluster
            x_min = x1[i] - window_size
            x_max = x1[i] + window_size
            y_min = y1[i] - window_size
            y_max = y1[i] + window_size
            in_window = np.bitwise_and(x1 >= x_min, x1 <= x_max)
            in_window = np.bitwise_and(
                in_window, np.bitwise_and(y1 >= y_min, y1 <= y_max)
            )
            in_window = np.bitwise_and(in_window, kot_arr[:, 3] == cluster_id)

            # Calculate and save the distances between the local maximas we find.
            if len(peaks1) > 0:
                this_point = [x1[i], y1[i]]
                peak_array = np.array(peaks1)
                this_point = np.array(this_point)
                this_point = this_point.reshape(1, 2)
                dist_peaks = distance.cdist(peak_array, this_point, "euclidean")

            # We find the local maximas for each window
            # Then we restric every local maximas that are way too close with each other with
            # the parameter "restrict_d". Then the local maximas with an accepted distace between
            # each other are relabeld as a unique number for each unique tree.
            if np.max(p1[in_window]) == p1[i] and np.min(dist_peaks) > restrict_d:
                peaks1.append([x1[i], y1[i]])
                points_to_label = np.argwhere(
                    np.logical_and(
                        np.abs(x1 - x1[i]) <= max_distance,
                        np.abs(y1 - y1[i]) <= max_distance,
                    )
                )
                points_to_label = points_to_label.flatten()
                if labels_k[i] == 0:
                    labels_k[points_to_label] = current_label
                    current_label += 1
                else:
                    labels_k[points_to_label] = labels_k[i]

        # we create a new array with the new labels for trunks
        new_2 = np.c_[x1, y1, z1, labels_k]
        if iteration == 0:
            final_result = new_2
        else:
            final_result = np.vstack((final_result, new_2))
        iteration = 1

    return final_result

In [34]:
# Find trees
full_array = np.column_stack((array_test, count))
Final_labels = cluster_local_maxima(full_array, window_size, max_distance, restrict_d)

In [35]:
# Get the number of trees in this las file
tree_count = np.unique(Final_labels[:, 3])
print("there are", len(tree_count), "trees in this area")

there are 742 trees in this area


In [36]:

# Initialize a list to hold the geometries for visualization
geometries = []

# Get the unique cluster labels
unique_labels = np.unique(Final_labels[:, 3])

# Iterate over each unique cluster label
for label in unique_labels:
    # Extract points belonging to the current cluster
    cluster_points = Final_labels[Final_labels[:, 3] == label][:, :3]
    
    # Create an Open3D point cloud object for the cluster
    cluster_pcd = o3d.geometry.PointCloud()
    cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)
    
    # Assign a random color to the cluster
    cluster_pcd.paint_uniform_color(np.random.rand(3))
    
    # Add the point cloud to the list of geometries
    geometries.append(cluster_pcd)
    
    # Create and add the bounding box for the cluster
    bbox = cluster_pcd.get_axis_aligned_bounding_box()
    bbox.color = (0, 0, 0)  # Set bounding box color (black)
    geometries.append(bbox)

# Visualize all clusters and their bounding boxes
o3d.visualization.draw_geometries(geometries, window_name='Clustered Trees with Bounding Boxes')

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is no

In [40]:
# Now calculate tree properties
segmented_trees = [Final_labels[Final_labels[:, 3] == label][:, :3] for label in unique_labels]

In [ ]:


# Filter out clusters that have fewer than 10 points
segmented_trees = [tree for tree in segmented_trees if tree.shape[0] >= 10]

def calculate_tree_top_height_and_location(tree_points):
    """
    Calculate the tree top height and location from a point cloud.
    """
    max_index = np.argmax(tree_points[:, 2])
    tree_top_height = tree_points[max_index, 2]
    tree_top_location = (tree_points[max_index, 0], tree_points[max_index, 1])
    
    return tree_top_height, tree_top_location

def calculate_crown_area(tree_points):
    """
    Calculate the crown area of a tree from its point cloud using a convex hull.
    """
    xy_points = tree_points[:, :2]
    
    if len(np.unique(xy_points, axis=0)) < 3:
        return 0
    
    hull = ConvexHull(xy_points)
    crown_area = hull.area
    
    return crown_area

def calculate_crown_diameter(crown_area):
    """
    Calculate the crown diameter given the crown area, assuming a circular shape.
    """
    crown_diameter = 2 * np.sqrt(crown_area / np.pi)
    return crown_diameter

def estimate_cbh(tree, plot=False):
    z_values = tree[:, 2]
    if len(z_values) < 4:
        return None

    sorted_indices = np.argsort(z_values)
    z_values_sorted = z_values[sorted_indices]
    
    percentile_ranking = np.linspace(0, 1, len(z_values_sorted))
    
    spline = UnivariateSpline(z_values_sorted, percentile_ranking, s=len(percentile_ranking))
    smoothed_curve = spline(z_values_sorted)
    
    first_derivative = spline.derivative(n=1)(z_values_sorted)
    second_derivative = spline.derivative(n=2)(z_values_sorted)
    
    inflection_points = np.where(np.diff(np.sign(second_derivative)))[0]
    valid_inflection_points = [i for i in inflection_points if percentile_ranking[i] < 0.5]
    
    if not valid_inflection_points:
        return None
    
    cbh_index = valid_inflection_points[np.argmax(first_derivative[valid_inflection_points])]
    cbh_height = z_values_sorted[cbh_index]
    
    second_quartile_height = np.percentile(z_values_sorted, 50)
    if cbh_height >= second_quartile_height:
        return None
    
    if plot:
        plot_percentile_profile(z_values_sorted, percentile_ranking, smoothed_curve, first_derivative, second_derivative, valid_inflection_points)
    
    return cbh_height

def calculate_ellipsoidal_volume(height, diameter):
    volume = (4/3) * np.pi * diameter * height
    return volume

# Initialize lists to collect data
tree_top_heights = []
tree_top_locations = []
cbh_heights = []
crown_areas = []
crown_diameters = []
crown_volumes = []

# Iterate over each tree and calculate properties
for tree_points in segmented_trees:
    # Calculate Tree Top Height and Location
    tree_top_height, tree_top_location = calculate_tree_top_height_and_location(tree_points)
    
    # Calculate CBH
    cbh_height = estimate_cbh(tree_points)
    if cbh_height is None:
        continue  # Skip this tree if CBH could not be estimated
    
    # Calculate Tree Crown Area
    crown_area = calculate_crown_area(tree_points)
    
    # Calculate Tree Crown Diameter
    crown_diameter = calculate_crown_diameter(crown_area)
    
    # Calculate the Crown Height
    crown_height = tree_top_height - cbh_height
    
    # Calculate Crown Volume
    crown_volume = calculate_ellipsoidal_volume(crown_height, crown_diameter)
    
    # Append results
    tree_top_heights.append(tree_top_height)
    tree_top_locations.append(tree_top_location)
    cbh_heights.append(cbh_height)
    crown_areas.append(crown_area)
    crown_diameters.append(crown_diameter)
    crown_volumes.append(crown_volume)

# Ensure all lists have the same length
num_trees = len(tree_top_heights)
assert len(tree_top_locations) == num_trees
assert len(cbh_heights) == num_trees
assert len(crown_areas) == num_trees
assert len(crown_diameters) == num_trees
assert len(crown_volumes) == num_trees

# Create a DataFrame to store the results
tree_data = pd.DataFrame({
    'Tree Top Location X (m)': [loc[0] for loc in tree_top_locations],
    'Tree Top Location Y (m)': [loc[1] for loc in tree_top_locations],
    'Tree Top Height (m)': tree_top_heights,
    'Canopy Base Height (CBH) (m)': cbh_heights,
    'Crown Area (sq m)': crown_areas,
    'Crown Diameter (m)': crown_diameters,
    'Crown Volume (cubic m)': crown_volumes
})

# Display the DataFrame
print(tree_data)


     Tree Top Location X (m)  Tree Top Location Y (m)  Tree Top Height (m)  \
0                186929.2380               428420.822              24.0450   
1                186967.7940               428423.854              28.2310   
2                186992.1030               428457.677               6.4240   
3                186982.8110               428431.644               5.3040   
4                186725.7110               428005.367               4.2910   
..                       ...                      ...                  ...   
289              186922.1460               428426.782              19.0940   
290              186942.8260               428418.781              27.2390   
291              186675.8470               428052.045              19.1140   
292              186963.9515               428415.981              27.6125   
293              186862.6190               428458.375              27.5710   

     Canopy Base Height (CBH) (m)  Crown Area (sq m)  Crown Dia

In [ ]:
def calculate_crown_area_polygon(tree_points):
    """Calculate the crown area as a polygon using the convex hull of the tree's 2D projection."""
    # Extract the x, y coordinates for the 2D projection
    xy_points = tree_points[:, :2]
    
    # Calculate the convex hull
    hull = ConvexHull(xy_points)
    
    # Create a polygon from the convex hull vertices
    hull_points = xy_points[hull.vertices]
    crown_polygon = Polygon(hull_points)
    
    return crown_polygon

# Prepare lists to store the results
crown_polygons = []
tree_top_heights = []
cbh_heights = []
crown_areas = []

# Iterate over all segmented trees
for tree_points in segmented_trees:
    # Calculate Tree Top Height
    tree_top_height, _ = calculate_tree_top_height_and_location(tree_points)
    tree_top_heights.append(tree_top_height)
    
    # # Estimate Canopy Base Height (CBH)
    # cbh_height = estimate_crown_base_height(tree_points)
    # cbh_heights.append(cbh_height)
    
    # Calculate Tree Crown Area Polygon
    crown_polygon = calculate_crown_area_polygon(tree_points)
    crown_polygons.append(crown_polygon)
    
    # Calculate Crown Area
    crown_areas.append(crown_polygon.area)

# Create a GeoDataFrame to store the crown polygons and attributes
gdf_crowns = gpd.GeoDataFrame({
    'Tree Top Height (m)': tree_top_heights,
    'Crown Area (sq m)': crown_areas,
    'geometry': crown_polygons
})

# Set the CRS to EPSG:7415 - Amersfoort / RD New + NAP height
gdf_crowns.set_crs(epsg=7415, inplace=True)

# Save the GeoDataFrame as a shapefile
output_shapefile = "tree_crowns_AHN4.shp"
gdf_crowns.to_file(output_shapefile)

print(f"Shapefile of tree crowns created at: {output_shapefile}")


Shapefile of tree crowns created at: tree_crowns_AHN4.shp


C:\Users\User\AppData\Local\Temp\ipykernel_14832\1551039068.py:56: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_crowns.to_file(output_shapefile)
C:\Users\User\anaconda3\envs\tree\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Tree Top Height (m)' to 'Tree Top H'
  ogr_write(
C:\Users\User\anaconda3\envs\tree\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Crown Area (sq m)' to 'Crown Area'
  ogr_write(


In [ ]:


# Create geometries for the GeoDataFrame
geometry = [Point(xy) for xy in zip(tree_data['Tree Top Location X (m)'], tree_data['Tree Top Location Y (m)'])]

# Create a GeoDataFrame with the updated column order
gdf = gpd.GeoDataFrame(tree_data, geometry=geometry)

# Set the CRS to EPSG:7415 - Amersfoort / RD New + NAP height
gdf.set_crs(epsg=7415, inplace=True)

# Save the GeoDataFrame as a shapefile
output_shapefile = "tree_tops_AHN4.shp"
gdf.to_file(output_shapefile)

print(f"Shapefile created at: {output_shapefile}")


Shapefile created at: tree_tops_AHN4.shp


C:\Users\User\AppData\Local\Temp\ipykernel_14832\3971038371.py:15: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shapefile)
C:\Users\User\anaconda3\envs\tree\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Tree Top Location X (m)' to 'Tree Top L'
  ogr_write(
C:\Users\User\anaconda3\envs\tree\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Tree Top Location Y (m)' to 'Tree Top_1'
  ogr_write(
C:\Users\User\anaconda3\envs\tree\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Tree Top Height (m)' to 'Tree Top H'
  ogr_write(
C:\Users\User\anaconda3\envs\tree\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Canopy Base Height (CBH) (m)' to 'Canopy Bas'
  ogr_write(
C:\Users\User\anaconda3\envs\tree\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normaliz